# Utilities

In [115]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import sklearn 
from skimage import transform
from skimage.transform import rotate, AffineTransform,warp
from skimage.util import random_noise
from skimage import util
from skimage.filters import gaussian
from sklearn import preprocessing
import numpy as np



import cv2
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt


In [116]:
def showImage(image):
    plt.imshow(image, cmap = matplotlib.cm.binary, interpolation='nearest')
    plt.axis('off')
    plt.show()

In [117]:

# image = cv2.imread("DataSet/trainData/club/2C0.jpg", cv2.COLOR_BGR2GRAY)
# showImage(image)

# img = image

# image = rotate(img, angle=5); 
# showImage(image)
# image = rotate(img, angle=-5); 
# showImage(image)
# image = transform.warp(img, AffineTransform(shear=-0.2), order=1, preserve_range=True, mode='wrap');
# showImage(image)
# image = transform.warp(img, AffineTransform(shear=0.1), order=1, preserve_range=True, mode='wrap'); 
# showImage(image)
# #----------------------------------------------------------------
# image = img.copy();
# image[:, 25:] = 0
# showImage(image)

# image = img.copy();
# image[:, 0:5] = 0
# showImage(image)

# image = img.copy();
# image[0:5,:] = 0
# showImage(image)

# image = img.copy();
# image[35:, :] = 0
# showImage(image)


In [118]:
# Image Augmentation, referncee: https://www.codespeedy.com/image-augmentation-using-skimage-in-python/

def augmentImage(img):
    augmentedImages=[img.flatten()]
    augmentedImages.append(img.flatten()) # add the image multiple times
    augmentedImages.append(img.flatten()) # add the image multiple times
    # rotation
    image = rotate(img, angle=5); augmentedImages.append(image.flatten())
    image = rotate(img, angle=-5); augmentedImages.append(image.flatten())
    # # Shearing
    image = transform.warp(img, AffineTransform(shear=-0.2), order=1, preserve_range=True, mode='wrap'); augmentedImages.append(image.flatten())
    image = transform.warp(img, AffineTransform(shear=0.1), order=1, preserve_range=True, mode='wrap'); augmentedImages.append(image.flatten())
    # # Cropping
    image = img.copy(); image[:, 23:] = 0; augmentedImages.append(image.flatten())
    image = img.copy(); image[:, 0:7] = 0; augmentedImages.append(image.flatten())
    image = img.copy(); image[0:7,:] = 0; augmentedImages.append(image.flatten())
    image = img.copy(); image[33:, :] = 0; augmentedImages.append(image.flatten())
    
    return augmentedImages
    
    


# CONSTANTS

In [119]:
CHARACTER_DIM=(30,30)
DIRECTORY=r'Arabic letters/Data Set'

# Import and split the data

In [120]:
 
import os
trainData = []
trainLabels = []


classes_dirs=os.listdir(DIRECTORY)

numClassesWithLimit = 0
LIMIT = 2000 # to have the same number of items in all classes
for innerDir in classes_dirs:
    count = 0
    for filename in os.listdir(DIRECTORY+'/' +innerDir):
        # read image
        image = cv2.imread(DIRECTORY+'/' + innerDir +'/' +filename, cv2.IMREAD_GRAYSCALE)/255.0
         
        image=cv2.resize(image,CHARACTER_DIM)

        augmentedImages = augmentImage(image.copy())
        count +=  len(augmentedImages)

        # append images
        trainData+=augmentedImages
        trainLabels+=[innerDir] * len(augmentedImages)

        # to break the inner loop if the number of items in the class exceeded the LIMIT
        if (count >= LIMIT):
            numClassesWithLimit+=1
            print("Class", innerDir , "has", count, "items")
            break

print("Number of classes that have exceeded", LIMIT, "items is:", numClassesWithLimit, "out of", len(os.listdir(DIRECTORY)))
print("trainData Size:", len(trainData), "and trainLabels size:", len(trainLabels))



Number of classes that have exceeded 2000 items is: 0 out of 28
trainData Size: 13541 and trainLabels size: 13541


In [121]:
# show_images(trainData)

In [122]:
from sklearn.model_selection import train_test_split

# split the data
trainData, testData, trainLabels, testLabels = train_test_split(trainData, trainLabels, test_size = 0.1 , random_state=50)


# Trainning

In [123]:
model_KNN = KNeighborsClassifier(n_neighbors=len(classes_dirs), weights='distance',n_jobs=-1) 
# print(trainData,trainLabels)
model_KNN.fit(trainData, trainLabels)



KNeighborsClassifier(n_jobs=-1, n_neighbors=28, weights='distance')

In [124]:
# from sklearn.svm import LinearSVC
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler

# model_SVC = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5, max_iter=1e4))
# # model_SVC = svm.SVC(gamma=.001,C=100)
# model_SVC.fit(trainData, trainLabels)

# Save the model

In [125]:
from joblib import dump, load
# save the model
dump(model_KNN, 'model.joblib') 

# load the model
# model = load('model.joblib')

model = model_KNN

# Evaluate the model

In [126]:
from sklearn.metrics import accuracy_score

# KNN
Labels_KNN_pred = model_KNN.predict(testData) 
print("KNN Model Accuarcy", accuracy_score(testLabels, Labels_KNN_pred)) # validate the model

KNN Model Accuarcy 0.9107011070110701


In [127]:
# from sklearn.metrics import accuracy_score

# # SVM
# Labels_SVM_pred = model_SVC.predict(testData) # test -predict the test trainData-
# print("SVM Model Accuarcy", accuracy_score(testLabels, Labels_SVM_pred)) # validate the model

# Random TestCases

In [131]:
import random 

count  = 0
LIMIT = 100
for i in range(LIMIT):
    randomIndex = random.randint(0,len(testData))
    print("Predicted", testLabels[randomIndex], "as", model_KNN.predict([testData[randomIndex]])[0])
    if (testLabels[randomIndex] == model_KNN.predict([testData[randomIndex]])[0]):
        count += 1
print(count, "Passed out of", LIMIT)
# print("Predicted", testLabels[randomIndex], "as", model.predict([testData[randomIndex]]),
#     "With confidence", model.predict_proba([testData[randomIndex]]))


Predicted 4 as 4
Predicted alf as alf
Predicted 3 as 3
Predicted 4 as 4
Predicted fa2 as lam
Predicted ya2 as ya2
Predicted ta2 as ta2
Predicted wow as wow
Predicted 1 as 1
Predicted wow as wow
Predicted 5 as dal
Predicted 1 as 1
Predicted 3 as 3
Predicted dal as dal
Predicted 6 as 6
Predicted dal as dal
Predicted gem as gem
Predicted kaf as kaf
Predicted dal as dal
Predicted non as non
Predicted ta2 as ta2
Predicted gem as gem
Predicted dal as dal
Predicted 9 as 9
Predicted 5 as 5
Predicted 4 as 4
Predicted 8 as 8
Predicted ta2 as dal
Predicted sad as sad
Predicted dal as heh
Predicted 6 as 6
Predicted 9 as 9
Predicted 4 as 4
Predicted non as non
Predicted kaf as kaf
Predicted 6 as 6
Predicted 8 as 8
Predicted beh as beh
Predicted 3 as 3
Predicted 1 as 1
Predicted gem as gem
Predicted non as non
Predicted beh as beh
Predicted 8 as 8
Predicted sad as sad
Predicted wow as wow
Predicted 1 as 1
Predicted 5 as 5
Predicted non as non
Predicted 8 as 8
Predicted 5 as 5
Predicted lam as kaf
Pr

# Testing on Input Folder

In [132]:
right=0
failed=[]
inputsDir='Arabic letters/input'
inputs=os.listdir(inputsDir)
for imgDir in inputs:
    imgName=(imgDir.split('/')[-1]).split('.')[0]
    oneImage = cv2.imread(inputsDir+'/'+imgDir,cv2.IMREAD_GRAYSCALE)/255
    oneImage = cv2.resize(oneImage,CHARACTER_DIM)
    
    predicted=model_KNN.predict([oneImage.flatten()])[0]

    if imgName.startswith(predicted):
        right+=1
    else:
        failed.append({
            'file':imgDir,
            'predictedAs':predicted
        })

print(f'predicted {right} out of {len(inputs)}')

print('FAILED: ', failed )


predicted 55 out of 55
FAILED:  []


# Compare between KNN and SVM

In [ ]:
# import os

# dire="dataSet/testData/"
# for filename in os.listdir(dire):
#     image = cv2.imread(dire+filename, cv2.COLOR_BGR2GRAY)/255.0
#     image.resize(40, 30)
#     # print("KNN:", model_KNN.predict([image.flatten()]), "SVM:", model_SVC.predict([image.flatten()]))
#     # showImage(image)
#     imageInverted = 1-image 
#     # print("SVM:", model_SVC.predict([imageInverted.flatten()]))
#     print("KNN:", model_KNN.predict([image.flatten()]), "SVM:", model_SVC.predict([image.flatten()]))
#     showImage(imageInverted)